In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from urllib.request import urlopen
import time
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import xlwt

driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get("https://ebook.kyobobook.co.kr/dig/etc/landing/best")

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.78M/6.78M [00:00<00:00, 10.2MB/s]
C:\Users\woosung\AppData\Local\Temp\ipykernel_4728\1083539102.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [2]:
book_list = []

genre_dict = { 
        18 : "경제경영", # 
        19 : "자기계발", # 경제/비즈니스
        20 : "시/에세이", # 
        21 : "인문", # 
        22 : "종교", #  x
        23 : "소설", # 
        24 : "국어/외국어", # 
        25 : "정치/사회", #  x
        26 : "역사/문화", #  x
        27 : "과학/공학", # 
        28 : "IT/프로그래밍", # 
        29 : "건강/의학", #  
        30 : "가정/생활/요리", # 
        31 : "여행/취미", # 
        32 : "예술/대중문화", # 
        33 : "유아",  # x
        34 : "아동", # x
        35 : "청소년", # x
        36 : "교재/수험서", #
        37 : "외국도서", #
        38 : "매거진" # x
}

smu_genre_dict = {
    
    1 : "외국어", # 24번 국어/외국어
    2 : "문학", # 23번 소설  #문학, 장르문학, 에세이/산문을 묶어야 할듯
    3 : "장르문학", # 23번 소설
    4 : "경제/비즈니스", # 18번 경제경영
    5 : "인문", # 21번 인문
    6 : "에세이/산문", # 20번 시/에세이 
    7 : "사회", # 25번 정치사회
    8 : "역사", # 26번 역사/문화
    9 : "가정/생활", # 30번 가정/생활/요리 27개건강/의학
    10 : "자연/과학", # 27번 과학/공학
    11 : "취미/여행", # 31번 여행/취미
    12 : "국외ebook", # 38번 외국도서
    13 : "문화/예술", # 32번 예술/대중문화
    14 : "상명ebook", # x
    15 : "컴퓨터/인터넷", # 28번 IT/프로그래밍
    16 : "요약본", # x
    17 : "대학교재", # 36번 교재/수험서
    18 : "강의지원도서", # 36번 교재/수험서
    19 : "수험서/자격증", #36번 교재/수험서
    20 : "종교/역학", # 22번 종교
    21 : "기타미분류", # x 
    22 : "어린이", # # 33번,34번,35번 유아/아동/청소년년
    23 : "연령별분류", # x
    24 : "EBSRADIO시리즈", # x
    25 : "잡지", # 38번 매거진
    26 : "오디오북", # x
    27 : "만화" # 
    
}

sm_100_genre_dict = {
    1 : "문학", #23
    2 : "에세이/산문", #20
    3 : "인문", #21
    4 : "경제/비즈니스", #18,19
    5 : "자연/과학", #27
    6 : "컴퓨터/인터넷", #28
    7 : "외국어", #24
    8 : "가정/생활", #29,30, 31, 32
    9 : "강의지원도서", #36
    10 : "국외ebook" #38
}

#100개씩 보기 선택
driver.find_element(By.ID,"ebookLanding_pageCnt-button").click()
time.sleep(0.5)
    
driver.find_element(By.ID,"ui-id-58").click()
time.sleep(0.5)

# 장르 선택
for genre_num in range(18,39):
    driver.find_element(By.ID,"ebookLanding_msc-button").click()
    time.sleep(0.5)
    
    driver.find_element(By.ID,f"ui-id-{genre_num}").click()
    time.sleep(0.5)
    
    bsObject = BeautifulSoup(driver.page_source, 'html.parser')
    
    books = bsObject.select('div.prodDt')
    rank = 0
    for book in books:
        
        rank = rank + 1# 순위
        
        title = book.select_one('strong > h3 > a').text  #책 제목
        
        author = book.select_one('p.prodDt_info > span').text  #저자
        
        publisher = book.select('p.prodDt_info > span')[1].text #출판사
        
        date = book.select('p.prodDt_info > span')[2].text  #등록일
        
        price=book.select_one('div.prodDt_price span b').text # 가격
        
        if(book.select_one('div.prodDt_review > p > b') == None) : #만약 별점이 없으면 0으로
            review = 0
            
        else :
            review = book.select_one('div.prodDt_review > p > b').text #별점
            
        genre=genre_dict[genre_num]
        
        book_list.append([title,rank,author,publisher,date,price,review,genre,genre])



In [3]:
df = pd.DataFrame(book_list,columns=["제목","kyobo_순위","kyobo_저자","kyobo_출판사","kyobo_발매일","kyobo_가격","kyobo_평점","kyobo_장르","상명대학교_학술정보관_장르"])
df

,제목,kyobo_순위,kyobo_저자,kyobo_출판사,kyobo_발매일,kyobo_가격,kyobo_평점,kyobo_장르,상명대학교_학술정보관_장르
0,부자 아빠 가난한 아빠. 1(20주년 특별 기념판),1,로버트 기요사키,민음인,2018.03.20 (종이책 출간 2018.02.22),"10,710",9.7,경제경영,경제경영
1,부자의 그릇,2,이즈미 마사토,다산북스,2020.12.21 (종이책 출간 2020.12.14),"9,450",9.2,경제경영,경제경영
2,레버리지,3,롭 무어,다산북스,2023.02.15 (종이책 출간 2023.02.15),"11,340",0,경제경영,경제경영
3,"뇌, 욕망의 비밀을 풀다",4,한스 게오르크 호이젤,비즈니스북스,2019.09.30 (종이책 출간 2019.10.04),"11,340",9.5,경제경영,경제경영
4,금리의 역습,5,에드워드 챈슬러,위즈덤하우스,2023.02.01 (종이책 출간 2023.01.11),"20,790",9.7,경제경영,경제경영
...,...,...,...,...,...,...,...,...,...
1355,릿터 38호,25,민음사 편집부,민음사,2022.10.26,"8,190",0,매거진,매거진
1356,맥심 MAXIM 2020. 12월호,26,맥심편집부,와이미디어,2020.12.02,"3,420",10,매거진,매거진
1357,맥심 MAXIM 2022.9월호,27,맥심편집부,와이미디어,2022.09.01,"4,050",10,매거진,매거진
1358,맥심 MAXIM 2022.7월호,28,맥심편집부,와이미디어,2022.07.01,"4,050",9.2,매거진,매거진


In [4]:
df.to_csv("kyobo_genre_best_0220.csv", index = False)

In [12]:
df = pd.read_csv("kyobo_genre_best_0220.csv")  #다시 크롤링 하기 귀찮을때

In [5]:
# 장르 상명대에게 맞추기
df_1 = df.replace({'상명대학교_학술정보관_장르' : {'경제경영':'경제/비즈니스','자기계발':'경제/비즈니스','시/에세이':'에세이/산문','소설':'문학','국어/외국어':'외국어','과학/공학':'자연/과학','IT/프로그래밍':'컴퓨터/인터넷','가정/생활/요리':'가정/생활','여행/취미':'가정/생활','예술/대중문화':'가정/생활','교재/수험서':'수험서/자격증','외국도서':'국외ebook'}})
df_1

,제목,kyobo_순위,kyobo_저자,kyobo_출판사,kyobo_발매일,kyobo_가격,kyobo_평점,kyobo_장르,상명대학교_학술정보관_장르
0,부자 아빠 가난한 아빠. 1(20주년 특별 기념판),1,로버트 기요사키,민음인,2018.03.20 (종이책 출간 2018.02.22),"10,710",9.7,경제경영,경제/비즈니스
1,부자의 그릇,2,이즈미 마사토,다산북스,2020.12.21 (종이책 출간 2020.12.14),"9,450",9.2,경제경영,경제/비즈니스
2,레버리지,3,롭 무어,다산북스,2023.02.15 (종이책 출간 2023.02.15),"11,340",0,경제경영,경제/비즈니스
3,"뇌, 욕망의 비밀을 풀다",4,한스 게오르크 호이젤,비즈니스북스,2019.09.30 (종이책 출간 2019.10.04),"11,340",9.5,경제경영,경제/비즈니스
4,금리의 역습,5,에드워드 챈슬러,위즈덤하우스,2023.02.01 (종이책 출간 2023.01.11),"20,790",9.7,경제경영,경제/비즈니스
...,...,...,...,...,...,...,...,...,...
1355,릿터 38호,25,민음사 편집부,민음사,2022.10.26,"8,190",0,매거진,매거진
1356,맥심 MAXIM 2020. 12월호,26,맥심편집부,와이미디어,2020.12.02,"3,420",10,매거진,매거진
1357,맥심 MAXIM 2022.9월호,27,맥심편집부,와이미디어,2022.09.01,"4,050",10,매거진,매거진
1358,맥심 MAXIM 2022.7월호,28,맥심편집부,와이미디어,2022.07.01,"4,050",9.2,매거진,매거진


In [7]:
df.to_csv("kyobo_genre_best_0220_smu_genre.csv", index = False)